<a href="https://colab.research.google.com/github/Tom-Jung/IO_with_python/blob/main/Ch5.%20%EC%82%B0%EC%97%85%EC%97%B0%EA%B4%80%ED%9A%A8%EA%B3%BC%20%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np   # Numpy import
import pandas as pd  # Pandas import

3부문 총, 국산, 수입거래표 및 고용표 불러오기

In [34]:
url_Tt_3 = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Total_matrix_3sectors.csv"  # 3부문의 총거래표 불러오기
Tt_3 = pd.read_csv(url_Tt_3)

url_Dt_3 = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Domestic_matrix_3sectors.csv"  # 3부문의 국산거래표 불러오기
Dt_3 = pd.read_csv(url_Dt_3)

url_It_3 = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Imported_matrix_3sectors.csv"  # 3부문의 수입거래표 불러오기
It_3 = pd.read_csv(url_It_3)

url_emp_3 = "https://raw.githubusercontent.com/Tom-Jung/IO_with_python/main/data/Emp_3_secs.csv"  # 3부문의 고용표 불러오기
Emp_3 = pd.read_csv(url_emp_3)

In [35]:
Tt = Tt_3.to_numpy()  # numpy로 불러오기
Dt = Dt_3.to_numpy()
It = It_3.to_numpy()
Em = Emp_3.to_numpy()

In [36]:
Tt_1 = Tt[0:6,1:13].astype(float)
Dm_1 = Dt[0:4,1:10].astype(float)
Im_1 = It[0:4,1:10].astype(float)
Ep_1 = Em[:,1:3]

In [37]:
Ct = Tt_1[-1,0:3]  # 산출액
Va = Tt_1[-2,0:3]  # 부가가치

In [38]:
Ct_m = np.tile(Ct, reps=[3,1]) # 산출액을 3번 행반복

In [39]:
At = (Tt_1[0:3,0:3]/Ct_m).round(3)   # 총투입계수
Ad = (Dm_1[0:3,0:3]/Ct_m).round(3)   # 국산 투입계수
Ai = (Im_1[0:3,0:3]/Ct_m).round(3)   # 수입 투입계수
Av = (Va/Ct).round(3).reshape([1,3])              # 부가가치율

print('총투입계수표 : \n', At,'\n')
print('국산투입계수표 : \n', Ad,'\n')
print('수입투입계수표 : \n', Ai,'\n')
print('부가가치율 : \n', Av)

df_At = pd.DataFrame(At, columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기
df_Ad = pd.DataFrame(Ad, columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기
df_Ai = pd.DataFrame(Ai, columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기
df_Av = pd.DataFrame(Av, columns = ['농산품', '공산품', '서비스'], index=['부가가치율'])                  #pandas로 내보내기

총투입계수표 : 
 [[0.067 0.021 0.004]
 [0.267 0.527 0.164]
 [0.117 0.143 0.283]] 

국산투입계수표 : 
 [[0.067 0.017 0.004]
 [0.25  0.328 0.132]
 [0.117 0.136 0.263]] 

수입투입계수표 : 
 [[0.    0.004 0.   ]
 [0.017 0.199 0.032]
 [0.    0.008 0.02 ]] 

부가가치율 : 
 [[0.55  0.309 0.548]]


##### 생산유발계수(Lf)

In [40]:
Lf = np.linalg.inv(np.identity(3)-Ad)  # 생산유발계수

Lf_rw_sm = Lf.sum(axis=0)  # 생산유발계수 행합
Lf_cl_sm = Lf.sum(axis=1)  # 생산유발계수 열합

Lf_bwd = (Lf.sum(axis=0)/Lf_rw_sm.mean()).astype(float).reshape([1,3])  #영향력 계수
Lf_fwd = (Lf.sum(axis=1)/Lf_cl_sm.mean()).astype(float).reshape([1,3])  #감응도 계수

print('생산유발계수 : \n', Lf.round(3),'\n')

print('생산유발계수 행합 : \n', Lf_rw_sm.round(3),'\n')
print('생산유발계수 열합 : \n', Lf_cl_sm.round(3),'\n')

print('품목별 영향력계수 : \n', Lf_bwd.round(3),'\n')
print('품목별 감음도계수: \n', Lf_fwd.round(3),'\n')

생산유발계수 : 
 [[1.081 0.03  0.011]
 [0.452 1.556 0.281]
 [0.255 0.292 1.411]] 

생산유발계수 행합 : 
 [1.789 1.878 1.703] 

생산유발계수 열합 : 
 [1.122 2.29  1.958] 

품목별 영향력계수 : 
 [[0.999 1.049 0.951]] 

품목별 감음도계수: 
 [[0.627 1.279 1.094]] 



In [41]:
df_Lf = pd.DataFrame(Lf.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기

#### 수입유발계수(Li)

In [42]:
Li = (Ai@Lf).round(3)  #수입유발계수
print('수입유발계수 : \n', Li)

수입유발계수 : 
 [[0.002 0.006 0.001]
 [0.117 0.32  0.101]
 [0.009 0.018 0.03 ]]


#### 부가가치유발계수(Lv)

In [43]:
Va_m = np.diagflat(Av)  # 부가가치율 대각행렬
Va_m

array([[0.55 , 0.   , 0.   ],
       [0.   , 0.309, 0.   ],
       [0.   , 0.   , 0.548]])

In [44]:
Lv = (Va_m@Lf).round(3)  #부가가치유발계수
print('부가가치유발계수 : \n', Lv)

부가가치유발계수 : 
 [[0.595 0.016 0.006]
 [0.14  0.481 0.087]
 [0.14  0.16  0.773]]


In [45]:
df_Lv = pd.DataFrame(Lv.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  # pandas로 내보내기

In [46]:
df_Li = pd.DataFrame(Li.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기

### 취업유발계수

In [47]:
Ei = ((Ep_1[:3,1]/Ct)*10).astype(float).round(3 ) # 10억원당 취업인원
Le = (np.diag(Ei)@Lf).round(3)  #취업유발계수
print('취업계수 : \n', Ei, '\n')
print('취업유발계수 : \n', Le)

취업계수 : 
 [21.167  2.165  7.932] 

취업유발계수 : 
 [[22.885  0.627  0.236]
 [ 0.979  3.37   0.609]
 [ 2.023  2.315 11.188]]


In [48]:
df_Le = pd.DataFrame(Le.round(3),columns = ['농산품', '공산품', '서비스'], index=['농산품', '공산품', '서비스'])  #pandas로 내보내기

In [49]:
# CSV파일로 결과 내보내기
df_At.to_csv('총투입계수.csv', encoding='utf_8-sig')
df_Ad.to_csv('국산투입계수.csv', encoding='utf_8-sig')
df_Ai.to_csv('수입투입계수.csv', encoding='utf_8-sig')

df_Lf.to_csv('생산유발계수.csv', encoding='utf_8-sig')
df_Lv.to_csv('부가가치유발계수.csv', encoding='utf_8-sig')
df_Li.to_csv('수입유발계수.csv', encoding='utf_8-sig')
df_Le.to_csv('취업유발계수.csv', encoding='utf_8-sig')

### 최종수요 항목별 유발효과

##### 생산유발효과

In [50]:
Yd = Dt[0:3,5:8]  # 국산최종수요...소비, 투자, 수출
Yi = It[0:3,5:8]  # 수입최종수요...소비, 투자, 수출
print('국산최종수요 : \n', Yd)
print('수입최종수요 : \n', Yi)

국산최종수요 : 
 [[15.0 -1.0 1.0]
 [138.0 106.0 623.0]
 [997.0 425.0 131.0]]
수입최종수요 : 
 [[3.0 1.0 0.0]
 [75.0 61.0 5.0]
 [35.0 7.0 0.0]]


In [51]:
Lfyd = (Lf@Yd).astype(float).reshape([3,3])   # 데이터타입을 float으로 지정하고 3x3행렬로 지정
Lvyd = (Lv@Yd).astype(float).reshape([3,3])   # 데이터타입을 float으로 지정하고 3x3행렬로 지정
Liyd = (Li@Yd).astype(float).reshape([3,3]) + Yi.astype(float).reshape([3,3]) # 데이터타입을 float으로 지정하고 3x3행렬로 지정

print('최종수요항목별 생산유발액 : \n', Lfyd)
print('최종수요항목별 부가가치유발액 : \n', Lvyd)
print('최종수요항목별 수입유발액 : \n', Liyd)


최종수요항목별 생산유발액 : 
 [[  31.44138837    6.80547184   20.99243501]
 [ 501.95397687  284.05112776 1006.96203598]
 [1450.39943459  630.15901436  366.89681383]]
최종수요항목별 부가가치유발액 : 
 [[ 17.115   3.651  11.349]
 [155.217  87.821 311.2  ]
 [794.861 345.345 201.083]]
최종수요항목별 수입유발액 : 
 [[  4.855   2.059   3.871]
 [221.612 137.728 217.708]
 [ 67.529  21.649  15.153]]


In [52]:
##### 최종수요항목별 유발계수 검증...최종수요항목별 생산유발계수=산출액, 최종수요항목별 부가가치유발계수=부가가치, 최종수요항목별 수입유발계수=수입 이 성립하여야 함

In [53]:
Lfyd_sm = Lfyd.sum(axis=1).astype(float).reshape([3,1])
Lvyd_sm = Lvyd.sum(axis=1).astype(float).reshape([3,1])
Liyd_sm = Liyd.sum(axis=1).astype(float).reshape([3,1])

print('최종수요항목별 생산유발액의 열합 : \n', Lfyd_sm)
print('최종수요항목별 부가가치액의 열합 : \n', Lvyd_sm)
print('최종수요항목별 수입유발액의 열합 : \n', Liyd_sm)

최종수요항목별 생산유발액의 열합 : 
 [[  59.23929522]
 [1792.96714061]
 [2447.45526277]]
최종수요항목별 부가가치액의 열합 : 
 [[  32.115]
 [ 554.238]
 [1341.289]]
최종수요항목별 수입유발액의 열합 : 
 [[ 10.785]
 [577.048]
 [104.331]]


###### 최종수요항목별 생산, 부가가치 및 수입유발계수

In [54]:
Yd_rw =np.concatenate((Yd,Yd.sum(axis=0).reshape([1,3])))  #국산최종수요항목(소비, 투자, 수출)  및 행합 (4x3)
Yi_rw =np.concatenate((Yi,Yi.sum(axis=0).reshape([1,3])))  #수입최종수요항목별(소비, 투자, 수출) 및 행합 (4x3)

Yd_cl =np.concatenate((Yd_rw,Yd_rw.sum(axis=1).reshape([4,1])), axis=1)  #국산최종수요항목 및 (소비, 투자, 수출) 행합과 열합(4x4)
Yi_cl =np.concatenate((Yi_rw,Yi_rw.sum(axis=1).reshape([4,1])), axis=1)  #수입최종수요항목 및 (소비, 투자, 수출) 행합과 열합(4x4)

# Lfyd_rw_sm = Lfyd.sum(axis=0) # 최종수요항목별(소비, 투자, 수출) 생산유발계수 행합
# Lvyd_rw_sm = Lvyd.sum(axis=0) # 최종수요항목별(소비, 투자, 수출) 부가가치유발계수 행합
# Liyd_rw_sm = Liyd.sum(axis=0) # 최종수요항목별(소비, 투자, 수출) 수입유발계수 행합

# Yd_cl_sm =Yd_rw_sm.sum(axis=1)  #국산최종수요항목별(소비, 투자, 수출) 열합
# Yi_cl_sm =Yi_rw_sm.sum(axis=1)  #수입최종수요항목별(소비, 투자, 수출) 열합

# Lfyd_cl_sm = Lfyd_rw_sm.sum(axis=1) # 최종수요항목별(소비, 투자, 수출) 생산유발계수 행합
# Lvyd_cl_sm = Lvyd_rw_sm.sum(axis=1) # 최종수요항목별(소비, 투자, 수출) 부가가치유발계수 행합
# Liyd_cl_sm = Liyd_rw_sm.sum(axis=1) # 최종수요항목별(소비, 투자, 수출) 수입유발계수 행합
Yi_cl

array([[3.0, 1.0, 0.0, 4.0],
       [75.0, 61.0, 5.0, 141.0],
       [35.0, 7.0, 0.0, 42.0],
       [113.0, 69.0, 5.0, 187.0]], dtype=object)

In [55]:
Lfyd_rw = np.concatenate((Lfyd, Lfyd.sum(axis=0).reshape([1,3]))) # 최종수요항목(소비, 투자, 수출) 및 생산유발액 행합(4x3)
Lvyd_rw = np.concatenate((Lvyd, Lvyd.sum(axis=0).reshape([1,3]))) # 최종수요항목(소비, 투자, 수출) 및 부가가치유발액 행합(4x3)
Liyd_rw = np.concatenate((Liyd, Liyd.sum(axis=0).reshape([1,3]))) # 최종수요항목(소비, 투자, 수출) 및 수입유발액 행합(4x3)

Lfyd_cl =np.concatenate((Lfyd_rw, Lfyd_rw.sum(axis=1).reshape([4,1])), axis=1)  # 국산최종수요항목별 생산유발액(소비, 투자, 수출) 행합고 열합(4x4)
Lvyd_cl =np.concatenate((Lvyd_rw, Lvyd_rw.sum(axis=1).reshape([4,1])), axis=1)  # 국산최종수요항목별 부가가치유발액(소비, 투자, 수출) 행합고 열합(4x4)
Liyd_cl =np.concatenate((Liyd_rw, Liyd_rw.sum(axis=1).reshape([4,1])), axis=1)  # 국산최종수요항목별 수입유발액(소비, 투자, 수출) 행합고 열합(4x4)

In [56]:
Lf_fd_m = (Lfyd_cl/Yd_cl).astype(float) # 최종수요항목별 생산유발계수
Lf_fd_m

array([[ 2.09609256, -6.80547184, 20.99243501,  3.94928635],
       [ 3.63734766,  2.67972762,  1.61631145,  2.06801285],
       [ 1.45476373,  1.48272709,  2.80073904,  1.57595316],
       [ 1.72503896,  1.73776531,  1.84748515,  1.76577482]])

In [57]:
Lv_fd_m = (Lvyd_cl[-1,:]/Yd_cl[-1,:]).reshape([1,4]).astype(float) # 최종수요항목별 부가가치유발계수
Lv_fd_m

array([[0.84103739, 0.82418302, 0.69355232, 0.79163943]])

In [58]:
Li_fd_m = (Liyd_cl[-1,:]/(Yd_cl[-1,:]+Yi_cl[-1,:])).reshape([1,4]).astype(float) # 최종수요항목별 수입유발계수
Li_fd_m

array([[0.23277593, 0.26950918, 0.31148947, 0.26398322]])

#### 최종수요 유발액과 유발계수 나타내기

In [59]:
Lfyd_ff = np.concatenate((Lfyd_cl,Lf_fd_m)) # 생산유발
Lvyd_ff = np.concatenate((Lvyd_cl,Lv_fd_m)) # 부가가치유발
Liyd_ff = np.concatenate((Liyd_cl,Li_fd_m)) # 수입유발

In [60]:
df_Lfyd_ff = pd.DataFrame(Lfyd_ff.round(3),
                         columns = ['소비', '투자', '수출', '열합'],
                         index=['농산품', '공산품', '서비스','유발액합','농산품', '공산품', '서비스','총유발계수',])
df_Lfyd_ff

,소비,투자,수출,열합
농산품,31.441,6.805,20.992,59.239
공산품,501.954,284.051,1006.962,1792.967
서비스,1450.399,630.159,366.897,2447.455
유발액합,1983.795,921.016,1394.851,4299.662
농산품,2.096,-6.805,20.992,3.949
공산품,3.637,2.680,1.616,2.068
서비스,1.455,1.483,2.801,1.576
총유발계수,1.725,1.738,1.847,1.766


In [61]:
df_Lvyd_ff = pd.DataFrame(Lvyd_ff.round(3),
                         columns = ['소비', '투자', '수출', '열합'],
                         index=['농산품', '공산품', '서비스','유발액합','총유발계수',])
df_Lvyd_ff


,소비,투자,수출,열합
농산품,17.115,3.651,11.349,32.115
공산품,155.217,87.821,311.200,554.238
서비스,794.861,345.345,201.083,1341.289
유발액합,967.193,436.817,523.632,1927.642
총유발계수,0.841,0.824,0.694,0.792


In [62]:
df_Liyd_ff = pd.DataFrame(Liyd_ff.round(3),
                          columns = ['소비', '투자', '수출', '열합'],
                          index=['농산품', '공산품', '서비스','유발액합','총유발계수',])
df_Liyd_ff

,소비,투자,수출,열합
농산품,4.855,2.059,3.871,10.785
공산품,221.612,137.728,217.708,577.048
서비스,67.529,21.649,15.153,104.331
유발액합,293.996,161.436,236.732,692.164
총유발계수,0.233,0.270,0.311,0.264


###### 영향력계수(후방연쇄효과) 및 감응도계수(전방연쇄효과)

In [63]:
Lf_rw_sm = Lf.sum(axis=0)  # 생산유발계수 행합
Lf_cl_sm = Lf.sum(axis=1)  # 생산유발계수 행합

Lf_bwd = (Lf.sum(axis=0)/Lf_rw_sm.mean()).astype(float).reshape([1,3])  #영향력 계수
Lf_fwd = (Lf.sum(axis=1)/Lf_cl_sm.mean()).astype(float).reshape([1,3])  #감응도 계수

print('품목별 영향력계수 : \n', Lf_bwd)
print('품목별 감음도계수: \n', Lf_fwd)

품목별 영향력계수 : 
 [[0.99930101 1.0492553  0.9514437 ]]
품목별 감음도계수: 
 [[0.62683887 1.27945533 1.0937058 ]]


In [64]:
# pandas로 변환기

df_Lf_bwd = pd.DataFrame(Lf_bwd.round(3), columns = ['농산품', '공산품', '서비스'], index=['영향력계수'])
df_Lf_fwd = pd.DataFrame(Lf_fwd.round(3), columns = ['농산품', '공산품', '서비스'], index=['영향력계수'])
df_Lf_fwd

,농산품,공산품,서비스
영향력계수,0.627,1.279,1.094


In [65]:
# csv 파일로 내보내기
df_Lfyd_ff.to_csv('최종수요항목별 생산유발.csv', encoding='utf_8-sig')
df_Lvyd_ff.to_csv('최종수요항목별 부가가치유발.csv', encoding='utf_8-sig')
df_Liyd_ff.to_csv('최종수요항목별 수입유발.csv', encoding='utf_8-sig')
df_Lf_bwd.to_csv('영향력계수.csv', encoding='utf_8-sig')
df_Lf_bwd.to_csv('감응도계수.csv', encoding='utf_8-sig')